In [2]:
#import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [11]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            # First Convolutional Block
            nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(96),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Second Convolutional Block
            nn.Conv2d(96, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Third Convolutional Block
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            # Fourth Convolutional Block
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            # Fifth Convolutional Block
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Dynamic computation of flattened size
        self._initialize_fc_layer(num_classes)

    def _initialize_fc_layer(self, num_classes):
        with torch.no_grad():
            sample_input = torch.randn(1, 3, 32, 32)
            feature_output = self.features(sample_input)
            flattened_size = feature_output.view(1, -1).shape[1]

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(flattened_size, 4096),
            nn.BatchNorm1d(4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [13]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

# Initialize Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AlexNet().to(device)
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [15]:
# Training Loop
num_epochs = 50
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {running_loss/100:.4f}')
            running_loss = 0.0

    # Validation Phase
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    avg_val_loss = val_loss / len(testloader)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {running_loss/len(trainloader):.4f}, '
          f'Val Loss: {avg_val_loss:.4f}, '
          f'Accuracy: {accuracy:.2f}%')

    # Learning Rate Scheduling
    scheduler.step(avg_val_loss)

    # Save Best Model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

print(f'Best Test Accuracy: {best_accuracy:.2f}%')


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch [1/50], Step [100/1563], Loss: 1.5186
Epoch [1/50], Step [200/1563], Loss: 1.4295
Epoch [1/50], Step [300/1563], Loss: 1.4427
Epoch [1/50], Step [400/1563], Loss: 1.3512
Epoch [1/50], Step [500/1563], Loss: 1.3550
Epoch [1/50], Step [600/1563], Loss: 1.2734
Epoch [1/50], Step [700/1563], Loss: 1.2373
Epoch [1/50], Step [800/1563], Loss: 1.2089
Epoch [1/50], Step [900/1563], Loss: 1.1996
Epoch [1/50], Step [1000/1563], Loss: 1.1912
Epoch [1/50], Step [1100/1563], Loss: 1.2070
Epoch [1/50], Step [1200/1563], Loss: 1.1507
Epoch [1/50], Step [1300/1563], Loss: 1.1446
Epoch [1/50], Step [1400/1563], Loss: 1.1481
Epoch [1/50], Step [1500/1563], Loss: 1.0957
Epoch [1/50], Train Loss: 0.0457, Val Loss: 1.1404, Accuracy: 58.43%
Epoch [2/50], Step [100/1563], Loss: 1.0718
Epoch [2/50], Step [200/1563], Loss: 1.0734
Epoch [2/50], Step [300/1563], Loss: 1.0294
Epoch [2/50], Step [400/1563], Loss: 1.0485
Epoch [2/50], Step [500/1563], Loss: 1.0546
Epoch [2/50], Step [600/1563], Loss: 1.0441
E

In [16]:
torch.save(model.state_dict(), "alexNet.pth")